In [ ]:
#import libraries
import tensorflow as tf
#import mtcnn
import numpy as np
import cv2
import copy
import PIL
import pickle

In [ ]:
#cropping using Caffe model and OpenCV DNN
faces_cropped=[]

#load trained model
modelFile = "drive/MyDrive/GP22/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "drive/MyDrive/GP22/deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

#load image
img = cv2.imread('drive/MyDrive/GP22/image4.jpg')
image_np=np.asarray(img)

#keep the original height and width, Caffe model require resizing to 300*300
h, w = img.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0,
(300, 300), (104.0, 117.0, 123.0))

#path the image to the model
net.setInput(blob)

#extract
faces = net.forward()

#crop faces from the image
for i in range(faces.shape[2]):
        confidence = faces[0, 0, i, 2]
        if confidence > 0.5:
            box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x1, y1) = box.astype("int")
            faces_cropped.append(image_np[y:y1,x:x1])


In [ ]:
#load FaceNet model
FaceNet=tf.keras.models.load_model('drive/MyDrive/GP22/facenet_keras.h5')

In [ ]:
#preprocess the image
#resize
for i in range(len(faces_cropped)):
  faces_cropped[i]=PIL.Image.fromarray((faces_cropped[i]*255).astype(np.uint8))
  faces_cropped[i]=faces_cropped[i].resize((160,160))
  faces_cropped[i]=np.asarray(faces_cropped[i])
  
  #cast as float
  faces_cropped[i]=faces_cropped[i].astype('float32')
  
  #standraliztion
  mean=np.mean(faces_cropped[i])
  std=np.std(faces_cropped[i])
  faces_cropped[i]=(faces_cropped[i]-mean)/std
  
  #expand batch dimension so tensorflow can accept it
  faces_cropped[i]=np.expand_dims(faces_cropped[i],axis=0)

In [ ]:
#predict the embeddings
embeddings=[]
for current_face in faces_cropped:
  results=FaceNet.predict(current_face)
  embeddings.append(results)

#creat dictionary with names
ref_embeddings_names={
    0:'Mustafa'
}

In [ ]:
#save the embeddings
np.save('ref_embeddings',embeddings)

with open('ref_embeddings_names.pkl', 'wb') as f:
    pickle.dump(ref_embeddings_names, f)